In [ ]:
!pip install pillow
!pip install pytesseract
!pip install opencv-python-headless
!pip install imutils
!pip install requests

In [ ]:
# in a terminal: `add-apt-repository ppa:alex-p/tesseract-ocr`  `apt update`
!apt install -y tesseract-ocr-grc

In [ ]:
from PIL import Image
import pytesseract
import argparse
import cv2 as cv
import os
import shutil
import imutils
import numpy as np
import re
import json

elastic_suffix = ['']

def ocrPlayerScreen(filename):
    filepath = 'matches/player menu/' + filename
    im_rgb = cv.imread(filepath)
    im_crop1 = im_rgb[13:180,170:310]

    im = cv.cvtColor(im_crop1, cv.COLOR_BGR2GRAY)
    im = cv.threshold(im,150,255,cv.THRESH_BINARY_INV)[1]
    
    lower = np.array([0])
    upper = np.array([15])
    shapeMask = cv.inRange(im, lower, upper)

    cnts = cv.findContours(shapeMask.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts.reverse()

    for c in cnts:
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.04 * peri, True)

        if len(approx) == 4:
            (x, y, w, h) = cv.boundingRect(approx)
            ar = w / float(h)
            
            if w > 7 and w < 13 and h <= 3 and ar >= 3.5:
                
                im_crop2 = im_rgb[y + 16:y + 60,x + 168:x+650]
                im_crop2 = cv.cvtColor(im_crop2, cv.COLOR_BGR2GRAY)
                im_crop2 = cv.threshold(im_crop2,150,255,cv.THRESH_BINARY_INV)[1]

                try:
                    text = pytesseract.image_to_string(im_crop2)
                    first = text.split('\n')[0]
                    ascii = first.encode('ascii', 'ignore').strip().split(' ')
                    sans_special = ''
                    for string in ascii:
                        if bool(re.match('[a-zA-Z0-9]+$', string)):
                            sans_special += string + ' '
                    stripped = sans_special.strip()
                    twitch_id = filename.split('.')[0]
                    if stripped:
                        if os.path.isdir('d2-stream-name-parser/' + stripped) == False:
                            os.mkdir('d2-stream-name-parser/' + stripped)
                        data = {}
                        try:
                            with open('d2-stream-name-parser/' + stripped + '/twitch.json') as f:
                                for i in f:
                                    data = json.loads(i)

                        except:
                            print('new file!')
                        if twitch_id in data:
                            data[twitch_id] += 1
                        else:
                            data[twitch_id] = 1
                        with open('d2-stream-name-parser/' + stripped + '/twitch.json', 'w+') as f:
                            f.write(json.dumps(data))
                except:
                    print('issue parsing text')
                break
    try:
        os.remove(filepath)
    except:
        print('error removing files')

for filename in os.listdir('matches/player menu'):
    if not filename.startswith('.'):
        ocrPlayerScreen(filename)